In [34]:
import pathlib
import polars as pl
import plotly.express as px

In [35]:
BASE_PATH = pathlib.Path.cwd().parent / "data" / "raw" / "parquet_files" / "train"

DEPOSIT_PATH = BASE_PATH / "train_deposit_1.parquet"

In [36]:
def get_transforms(data: pl.LazyFrame) -> list[pl.Expr]:
    transforms = []
    for col, dtype in zip(data.columns, data.dtypes):
        if col.endswith("D"):
            transforms.append(
                pl.when(pl.col(col).is_null()).then(pl.lit(False)).otherwise(pl.lit(True)).cast(pl.Boolean).alias(col)
            )
        elif dtype == pl.String and not col.endswith("D"):
            transforms.append(
                pl.col(col).fill_null("UNKNOWN").cast(pl.Categorical)
            )
        elif dtype in [pl.Float64, pl.Float32]:
            transforms.append(
                pl.col(col).fill_null(pl.col(col).median()).cast(pl.Float32)
            )
        elif col in {}:
            transforms.append(
                pl.col(col).fill_null(pl.col(col).median()).shrink_dtype()
            )
        elif col in {}:
            transforms.append(
                pl.col(col).fill_null(pl.lit(False)).shrink_dtype()
            )
    return transforms


In [37]:
deposit_data: pl.LazyFrame = pl.scan_parquet(DEPOSIT_PATH)
deposit_data.sort("case_id", "num_group1").head().collect()

case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
i64,f64,str,i64,str
225,0.0,null,0,"""2016-08-16"""
331,260.374,"""2018-03-18""",0,"""2015-03-19"""
358,0.0,null,0,"""2014-09-02"""
390,211748.53,"""2017-07-22""",0,"""2014-07-23"""
390,203.602,"""2017-09-30""",1,"""2015-10-01"""


In [38]:
deposit_data.describe()

statistic,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
str,f64,f64,str,f64,str
"""count""",145086.0,145086.0,"""65404""",145086.0,"""145086"""
"""null_count""",0.0,0.0,"""79682""",0.0,"""0"""
"""mean""",1.4662e6,8422.304482,null,0.522531,null
"""std""",886528.958911,86232.120476,null,1.620954,null
"""min""",225.0,-40000.0,"""2002-02-27""",0.0,"""2001-11-19"""
"""25%""",660041.0,0.0,null,0.0,null
"""50%""",1.556939e6,223.658,null,0.0,null
"""75%""",2.530539e6,478.34,null,1.0,null
"""max""",2.703453e6,1.2213286e7,"""2020-03-18""",64.0,"""2017-07-31"""


In [39]:
deposit_data = deposit_data.with_columns(
    pl.when(pl.col("contractenddate_991D").is_not_null()).then(pl.lit(True)).otherwise(pl.lit(False)).alias("is_deposit_contract_ended")
)

In [40]:
deposit_data = deposit_data.group_by("case_id", "is_deposit_contract_ended").agg(
    pl.col("amount_416A").mean()
)

deposit_data.sort("case_id").head().collect()

case_id,is_deposit_contract_ended,amount_416A
i64,bool,f64
225,false,0.0
331,true,260.374
358,false,0.0
390,false,223.68001
390,true,105976.066
